<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Attempting-Networkx-to-Neo4J-approach" data-toc-modified-id="Attempting-Networkx-to-Neo4J-approach-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Attempting Networkx-to-Neo4J approach</a></span><ul class="toc-item"><li><span><a href="#two-concerns:" data-toc-modified-id="two-concerns:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>two concerns:</a></span><ul class="toc-item"><li><span><a href="#i'll-look-into-Py2Neo-for-now,-since-it-has-more-support" data-toc-modified-id="i'll-look-into-Py2Neo-for-now,-since-it-has-more-support-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>i'll look into Py2Neo for now, since it has more support</a></span></li></ul></li></ul></li></ul></div>

In [2]:
import networkx as nx
import neonx
import json

# Attempting Networkx-to-Neo4J approach

In [4]:
%pwd

'/Users/mark/Documents/flatiron/projects/sentimental/Sentimental'

In [8]:
with open('/Users/mark/Documents/flatiron/projects/sentimental/Sentimental/tweets.jsonl') as f:
    lines = f.readlines()
len(lines) # 29k tweets, each line in .jsonl is a new json

29061

In [11]:
jsons = []
for line in lines:
    js = json.loads(line) # .loads is different from .load 
    jsons.append(js) #.loads(tring), perhaps
len(jsons)

29061

In [14]:
j = jsons[0]

In [15]:
j.keys()

dict_keys(['timestamp', 'text', 'entities', 'lang', 'retweeted_status', 'quoted_status', 'retweet_count', 'favorite_count', 'user_id', 'coordinates', 'id', 'user'])

In [18]:
j['text']

'RT @zachjcarter: CHILDCARE\nBernie: universal\nWarren: means tested\n\nSTUDENT DEBT FORGIVENESS\nBernie: universal\nWarren: means tested\n\nRENT CO…'

In [19]:
j['entities']

{'hashtags': [],
 'urls': [],
 'user_mentions': [{'screen_name': 'zachjcarter',
   'name': 'Zach Carter',
   'id': 755826044225986560,
   'id_str': '755826044225986560',
   'indices': [3, 15]}],
 'symbols': []}

In [21]:
j['coordinates'] # some don't have geoloc

In [27]:
c = 0
for js in jsons:
    if js['coordinates'] == None:
        c+=1
print(c) # we've got 2/29,000 with geoloc. hmm

29059


In [28]:
def status_to_dict(tweet):
    try:
        if tweet.lang == 'in':
            return None
        tweet_ = dict()
        tweet_['timestamp'] = tweet.created_at.timestamp()
        if 'extended_text' in tweet._json.keys():
            tweet_['text'] = tweet.extended_text.full_text
            if 'extended_entities' in tweet.extended_text._json.keys():
                tweet_['entities'] = tweet.extended_text.entities
            else:
                tweet_['entities'] = tweet.extended_text.entities
        else:
            tweet_['text'] = tweet.text
            tweet_['entities'] = tweet.entities
        if tweet.lang:
            tweet_['lang'] = tweet.lang
        if 'retweeted_status' in tweet._json.keys():
            tweet_['retweeted_status'] = status_to_dict(tweet.retweeted_status)
        if 'quoted_status' in tweet._json.keys():
            tweet_['quoted_status'] = status_to_dict(tweet.quoted_status)
        if tweet.in_reply_to_status_id:
            tweet_['in_reply_to_status_id'] = tweet.in_reply_to_status_id
        if tweet.in_reply_to_user_id:
            tweet_['in_reply_to_user_id'] = tweet.in_reply_to_user_id
        if tweet.retweet_count:
            tweet_['retweet_count'] = tweet.retweet_count
        else:
            tweet_['retweet_count'] = 0
        if tweet.favorite_count:
            tweet_['favorite_count'] = tweet.favorite_count
        else:
            tweet_['favorite_count'] = 0
        tweet_['user_id'] = tweet.user.id
        tweet_['coordinates'] = tweet.coordinates
        tweet_['id'] = int(tweet.id)
    except Exception as e:
        print(e)
        logging.error(f'Error on status_to_dict[Tweet]: {e}\nFailed tweet: {tweet._json}\n')

    try:
        user = dict()
        user['screen_name'] = tweet.user.screen_name
        user['followers_count'] = tweet.user.followers_count
        user['verified'] = tweet.user.verified
        user['created_at'] = tweet.user.created_at.timestamp()
        user['id'] = tweet.user.id
        if tweet.user.lang:
            user['lang'] = tweet.user.lang
        tweet_['user'] = user
    except Exception as e:
        print(e)
        logging.error(f'Error on status_to_dict[User]: {e}\nFailed tweet: {tweet._json}\n')
        tweet_['user'] = None
    return tweet_

In [31]:
j['lang']

'en'

In [32]:
# parsed = []
# for j in jsons:
#     out = status_to_dict(j)
#     parsed.append(out)

## two concerns:
- rebuilding Dan's methods to fit into a networkx graph would require a lot of strange resyntaxing, so 'if it ain't broke'
- networkx-neo4j only supports one type/label of edge per graph write-push
    - this tweet graph has nodes of several types interlocking with edges of several types
    - no clear way to composite push the graph without duplication?
### i'll look into Py2Neo for now, since it has more support